In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '1', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/topic_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 1000, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 20, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

In [8]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    

# run model 

## init

In [9]:
def get_feed_dict(batch, mode='train'):
    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    keep_prob = config.keep_prob if mode == 'train' else 1.0
    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [24]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow])
t_variables['keep_prob'] = tf.placeholder(tf.float32)

tree_idxs = {0:[1, 2], 
              1:[10, 11], 2:[20, 21, 22], 
              10: [100, 101], 11: [110, 111, 112], 20: [200, 201], 21: [210, 211], 22:[220, 221, 222]
              }

# tree_idxs = {0:[1, 2], 
#                       1:[10, 11], 2:[20, 21, 22]}

topic_idxs = [0] + [idx for child_idxs in tree_idxs.values() for idx in child_idxs]

## doubly rnn

In [25]:
class DoublyRNNCell:
    def __init__(self, dim_hidden, output_layer=None):
        self.dim_hidden = dim_hidden
        
        self.ancestral_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='ancestral')
        self.fraternal_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='fraternal')
        self.hidden_layer = tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='hidden')
        
        self.output_layer=output_layer
        
    def __call__(self, state_ancestral, state_fraternal, reuse=True):
        with tf.variable_scope('input', reuse=reuse):
            state_ancestral = self.ancestral_layer(state_ancestral)
            state_fraternal = self.fraternal_layer(state_fraternal)

        with tf.variable_scope('output', reuse=reuse):
            state_hidden = self.hidden_layer(state_ancestral + state_fraternal)
            if self.output_layer is not None: 
                output = self.output_layer(state_hidden)
            else:
                output = state_hidden
            
        return output, state_hidden
    
    def get_initial_state(self, name):
        initial_state = tf.get_variable(name, [1, self.dim_hidden], dtype=tf.float32)
        return initial_state
    
    def get_zero_state(self, name):
        zero_state = tf.zeros([1, self.dim_hidden], dtype=tf.float32, name=name)
        return zero_state

In [26]:
def doubly_rnn(dim_hidden, tree_idxs, initial_state_parent=None, initial_state_sibling=None, output_layer=None, name=''):
    outputs, states_parent = {}, {}
    
    with tf.variable_scope(name, reuse=False):
        doubly_rnn_cell = DoublyRNNCell(dim_hidden, output_layer)

        if initial_state_parent is None: initial_state_parent = doubly_rnn_cell.get_initial_state('init_state_parent')
        if initial_state_sibling is None: 
#             initial_state_sibling = doubly_rnn_cell.get_initial_state('init_state_sibling')
            initial_state_sibling = doubly_rnn_cell.get_zero_state('init_state_sibling')
        output, state_sibling = doubly_rnn_cell(initial_state_parent, initial_state_sibling, reuse=False)
        outputs[0], states_parent[0] = output, state_sibling

        for parent_idx, child_idxs in tree_idxs.items():
            state_parent = states_parent[parent_idx]
            state_sibling = initial_state_sibling
            for child_idx in child_idxs:
                output, state_sibling = doubly_rnn_cell(state_parent, state_sibling)
                outputs[child_idx], states_parent[child_idx] = output, state_sibling

    return outputs

## stick break

In [27]:
def hierarchical_sbp(tree_sticks_topic, tree_sticks_branch):
    tree_prob_topic = {}
    rest_topics = {}

    # calculate topic probability and save
    stick_topic = tree_sticks_topic[0]
    tree_prob_topic[0] = stick_topic
    rest_topics[0] = 1.-stick_topic
    for parent_idx, child_idxs in tree_idxs.items():
        rest_topic = rest_topics[parent_idx]
        rest_branch = 1.
        for child_idx in child_idxs:
            # calculate topic probability
            if child_idx == child_idxs[-1]: # last child
                prob_branch = rest_branch # phi
            else:
                stick_branch = tree_sticks_branch[child_idx] # psi
                prob_branch = stick_branch * rest_branch # phi

            if not child_idx in tree_idxs: # leaf childs
                prob_topic = prob_branch * rest_topic # pi
            else:
                stick_topic = tree_sticks_topic[child_idx] # upsilon
                prob_topic = stick_topic * prob_branch * rest_topic # pi

            # save topic probability and update rest stick length
            tree_prob_topic[child_idx] = prob_topic
            rest_branch = (1.- stick_branch) * rest_branch
            rest_topics[child_idx] = (1.-stick_topic)*prob_branch * rest_topic
            
    return tree_prob_topic

## build model

In [28]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_bow')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors
    prob_layer = lambda h: tf.nn.sigmoid(tf.matmul(latents_bow, h, transpose_b=True))
    
    tree_sticks_topic = doubly_rnn(config.dim_latent_bow, tree_idxs, output_layer=prob_layer, name='sticks_topic')
    tree_sticks_branch = doubly_rnn(config.dim_latent_bow, tree_idxs, output_layer=prob_layer, name='sticks_branch')

    tree_prob_topic = hierarchical_sbp(tree_sticks_topic, tree_sticks_branch)
    prob_topic = tf.concat(list(tree_prob_topic.values()), 1)

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    tree_topic_embeddings = doubly_rnn(config.dim_emb, tree_idxs, name='emb_topic')
    topic_embeddings = tf.concat(list(tree_topic_embeddings.values()), 0)
    
    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

## define loss

In [29]:
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(len(topic_idxs))))

global_step = tf.Variable(0, name='global_step',trainable=False)

loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)

grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]
opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

In [30]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, ppl_mean

def print_topic_sample():
    topics_freq_bow_idxs = bow_idxs[sess.run(topics_freq_bow_indices)]
    assert len(topics_freq_bow_idxs) == len(topic_idxs)
    for topic_idx, topic_freq_bow_idxs in zip(topic_idxs, topics_freq_bow_idxs):
        print(topic_idx, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))

In [31]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','VALID:','TM','','',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG']]))))

In [ ]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log = sess.run(tf.train.get_global_step())
            
#             if loss_dev < loss_min:
#                 loss_min = loss_dev
#                 saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            
            # visualize topic
            print_topic_sample()

            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                      VALID:    TM          \
      Time  Ep    Ct    LOSS   PPL     NLL    KL   REG    LOSS   PPL     NLL   
1        2   0     0  123.51  1035  121.98  0.58  0.95  117.01  1015  115.75   
1001    18   0  1000  124.40   973  122.89  1.40  0.11  108.77   617  107.75   
2001    18   0  2000  120.23   780  118.97  1.16  0.10  107.29   575  106.31   
2276     5   1     0  119.54   754  118.29  1.15  0.10  107.10   565  105.96   
3276    19   1  1000  118.00   694  116.72  1.20  0.09  106.66   543  105.44   
4276    19   1  2000  117.10   659  115.79  1.23  0.08  106.35   536  105.16   
4551     5   2     0  116.92   652  115.61  1.24  0.08  106.37   533  105.19   
5551    19   2  1000  116.32   631  114.98  1.26  0.07  105.86   520  104.60   
6551    19   2  2000  115.92   617  114.57  1.28  0.07  105.93   524  104.69   
6826     5   3     0  115.85   613  114.49  1.29  0.07  106.44   543  105.22   
7826    20   3  1000  115.53   602  114.16  1.30  0.07  105.83   519  104.59   
8826    20   3  2000  115.30   594  113.91  1.32  0.06  105.68   509  104.48   
9101     6   4     0  115.23   591  113.84  1.33  0.06  105.65   510  104.43   
10101   19   4  1000  115.06   584  113.66  1.34  0.06  105.65   514  104.36   
11101   22   4  2000  114.84   578  113.43  1.35  0.06  105.64   508  104.37   
11376    6   5     0  114.81   577  113.40  1.36  0.06  105.63   509  104.32   
12376   21   5  1000  114.67   572  113.24  1.37  0.06  107.42   570  106.26   
13376   21   5  2000  114.54   569  113.14  1.35  0.06  105.14   505  104.21   
13651    5   6     0  114.51   568  113.11  1.34  0.06  105.28   509  104.33   
14651   20   6  1000  114.41   564  113.02  1.33  0.06  104.84   497  103.89   
15651   21   6  2000  114.23   560  112.85  1.32  0.06  104.62   487  103.60   
15926    4   7     0  114.19   559  112.81  1.32  0.06  104.96   497  103.93   
16926   16   7  1000  114.01   555  112.64  1.32  0.05  104.71   491  103.67   
17926   17   7  2000  113.94   551  112.57  1.32  0.05  104.38   483  103.33   
18201    5   8     0  113.91   550  112.54  1.31  0.05  104.41   479  103.35   
19201   17   8  1000  113.81   547  112.44  1.31  0.05  104.62   483  103.47   
20201   18   8  2000  113.70   543  112.33  1.31  0.05  104.44   481  103.36   
20476    5   9     0  113.67   543  112.30  1.31  0.05  104.18   473  103.11   
21476   18   9  1000  113.58   540  112.21  1.31  0.05  104.33   476  103.17   
22476   19   9  2000  113.48   537  112.11  1.32  0.05  104.23   471  103.12   
22751    5  10     0  113.46   536  112.09  1.32  0.05  104.37   476  103.18   
23751   20  10  1000  113.38   534  112.02  1.32  0.05  104.18   468  103.05   
24751   20  10  2000  113.30   531  111.93  1.32  0.05  104.14   466  102.96   
25026    5  11     0  113.27   531  111.91  1.32  0.05  104.18   471  102.95   
26026   21  11  1000  113.19   528  111.82  1.32  0.05  103.73   458  102.55   
27026   20  11  2000  113.14   526  111.77  1.33  0.05  104.00   464  102.77   
27301    6  12     0  113.11   526  111.74  1.33  0.05  104.09   466  102.88   
28301   20  12  1000  113.06   524  111.68  1.33  0.05  104.07   468  102.90   
29301   20  12  2000  113.00   522  111.62  1.33  0.04  104.10   468  102.90   
29576    5  13     0  112.97   521  111.60  1.33  0.04  104.02   466  102.81   
30576   20  13  1000  112.93   520  111.55  1.33  0.04  103.93   464  102.73   
31576   22  13  2000  112.87   518  111.49  1.34  0.04  103.96   461  102.74   
31851    6  14     0  112.85   518  111.47  1.34  0.04  104.04   466  102.86   
32851   21  14  1000  112.79   516  111.40  1.34  0.04  103.83   457  102.55   
33851   20  14  2000  112.74   514  111.35  1.35  0.04  103.93   462  102.65   
34126    6  15     0  112.74   514  111.34  1.35  0.04  104.00   463  102.70   
35126   20  15  1000  112.69   512  111.29  1.36  0.04  103.82   454  102.50   

                   
         KL   REG  
1      0.31  0.95  
1001   0.95  0.08  
2001   0.89  0.08  
2

0  BOW: ! sleeve ; & price love bought nice recommend perfect
1  BOW: color pro love mac put nice cover easy easily hard
2  BOW: room pocket pockets small extra space hold charger accessories front
10  BOW: bottom cover top protector screen plastic place back side shell
11  BOW: back quality 've zipper years bought strap time months ...
20  BOW: carry power accessories card drive big storage tablet padding ipad
21  BOW: quality work bags carry compartments stuff travel time big pockets
22  BOW: carry pockets compartment stuff compartments plenty work items storage shoulder
100  BOW: cover keyboard hard easy purple clean apple blue white logo
101  BOW: back problem mine speck part cheap money year weeks thing
110  BOW: months corners cracked dropped guess corner piece remove plastic fell
111  BOW: return shipping money received service amazon customer ago refund cheap
112  BOW: months straps broke targus week started shoulder year weeks handle
200  BOW: snug perfectly surface picture pr

# confirm variables

In [105]:
debug_value([topic_states_parent[:, :5]])

strided_slice_3:0 : [[ 0.0228974  -0.00080273 -0.00123198  0.00688633  0.02475363]
 [-0.03019566  0.0189551   0.01089677 -0.01826636  0.00705749]
 [-0.03019566  0.0189551   0.01089677 -0.01826636  0.00705749]
 [-0.08289671  0.18277843 -0.01282167 -0.00955842 -0.0806391 ]
 [-0.08289671  0.18277843 -0.01282167 -0.00955842 -0.0806391 ]
 [-0.08289671  0.18277843 -0.01282167 -0.00955842 -0.0806391 ]
 [-0.08289671  0.18277843 -0.01282167 -0.00955842 -0.0806391 ]
 [-0.08289671  0.18277843 -0.01282167 -0.00955842 -0.0806391 ]
 [-0.1493729  -0.12025591 -0.11520056 -0.03578362  0.21700668]
 [-0.1493729  -0.12025591 -0.11520056 -0.03578362  0.21700668]
 [-0.1493729  -0.12025591 -0.11520056 -0.03578362  0.21700668]
 [-0.1493729  -0.12025591 -0.11520056 -0.03578362  0.21700668]
 [-0.1493729  -0.12025591 -0.11520056 -0.03578362  0.21700668]
 [-0.1493729  -0.12025591 -0.11520056 -0.03578362  0.21700668]
 [-0.1493729  -0.12025591 -0.11520056 -0.03578362  0.21700668]
 [-0.1493729  -0.12025591 -0.115200

In [101]:
tree_idxs

{0: [1, 2],
 1: [10, 11],
 2: [20, 21, 22],
 10: [100, 101],
 11: [110, 111, 112],
 20: [200, 201],
 21: [210, 211],
 22: [220, 221, 222]}

In [ ]:
debug_value([tf.exp(-tf.divide(topic_losses_recon, n_bow))])

### test

In [12]:
debug_shape([bow, hidden_bow, latents_bow, prob_topic, bow_embeddings, topic_embeddings, topic_bow, prob_bow])

Placeholder:0 : (32, 4022)
topic/enc/dropout/cond/Merge:0 : (32, 256)
topic/enc/add:0 : (32, 32)
topic/enc/prob/Softmax:0 : (32, 50)
embedding_lookup:0 : (4022, 256)
topic/dec/topic_emb:0 : (50, 256)
topic/dec/Softmax:0 : (50, 4022)
topic/dec/Log:0 : (32, 4022)


In [27]:
debug_shape([topic_losses_recon, topic_loss_recon, n_bow, ppls, topic_embeddings_norm, tf.expand_dims(topic_angles_mean, -1), topic_angles_vars])

Sum:0 : (32,)
Neg:0 : ()
Sum_2:0 : (32,)
Neg_1:0 : (32,)
truediv_1:0 : (50, 256)
ExpandDims_1:0 : (1,)
Mean_3:0 : ()


In [14]:
debug_value([tf.reduce_sum(tf.square(topic_embeddings_norm), 1)], return_value=True)[0]

array([1.        , 0.99999994, 0.9999999 , 1.        , 1.        ,
       1.0000001 , 0.9999999 , 1.0000001 , 1.0000001 , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.9999999 , 0.9999999 , 0.99999994, 1.        ,
       1.        , 0.9999999 , 1.0000001 , 1.        , 1.        ,
       1.        , 0.99999994, 1.        , 0.99999994, 0.99999994,
       1.0000001 , 0.9999999 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.9999999 , 1.        ,
       0.9999999 , 1.        , 1.        , 1.        , 0.99999994,
       1.0000001 , 1.        , 1.        , 0.99999994, 1.        ],
      dtype=float32)

In [15]:
debug_value([tf.reduce_sum(prob_topic, -1), tf.reduce_sum(topic_bow, -1), tf.reduce_sum(tf.exp(prob_bow), 1)])

Sum_4:0 : [1.0000001  1.         1.0000001  0.99999994 1.         1.
 1.         0.99999994 0.99999994 1.         1.         1.
 0.99999994 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.9999999
 1.         0.99999994 1.         1.0000001  0.99999994 1.
 1.         1.        ]
Sum_5:0 : [1.         0.99999994 1.         0.99999994 0.99999994 1.
 1.         0.9999998  1.         1.         1.         1.
 1.         1.         1.         1.0000001  0.99999994 1.
 1.         0.99999994 0.9999999  0.99999994 1.0000001  1.
 1.         1.         1.         0.99999994 0.9999999  1.
 1.         1.         0.99999994 1.         0.99999994 1.
 1.         0.9999999  1.         1.         1.         1.
 0.99999994 1.         0.99999994 0.99999994 0.99999994 0.99999994
 1.         1.        ]
Sum_6:0 : [1.         1.         1.0000001  0.99999994 1.         1.
 0.9999999  0.99999994 0.9999999  1.         1.0000001  1.
 0.9999999  1.        

In [16]:
sigma_bow = tf.exp(0.5 * logvars_bow)
dist_bow = tfd.Normal(means_bow, sigma_bow)
dist_std = tfd.Normal(0., 1.)
topic_loss_kl_tmp = tf.reduce_mean(tf.reduce_sum(tfd.kl_divergence(dist_bow, dist_std), 1))

In [17]:
debug_value([topic_loss_recon, topic_loss_kl, topic_loss_kl_tmp])

Neg:0 : 405.38312
Mean_1:0 : 0.32056683
Mean_4:0 : 0.32056683


In [110]:
_logvars, _means, _kl_losses, _latents, _output_logits = sess.run([logvars, means, kl_losses, latents, output_logits], feed_dict=feed_dict)


In [111]:
_logvars.shape, _means.shape, _kl_losses.shape, _latents.shape

((32, 32), (32, 32), (32,), (32, 32))

In [112]:
_output_logits

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)

In [109]:
_output_logits, _dec_target_idxs_do, _dec_mask_tokens_do, _recon_loss, _kl_losses, _ = sess.run([output_logits, dec_target_idxs_do, dec_mask_tokens_do, recon_loss, kl_losses, opt], feed_dict=feed_dict)


NameError: name 'dec_target_idxs_do' is not defined

In [44]:
tf.reduce_max(output_logits, 2).eval(session=sess, feed_dict=feed_dict).shape

(120, 46)

In [31]:
_output_logits.shape, _dec_target_idxs_do.shape, _dec_mask_tokens_do.shape

((120, 46, 20000), (120, 46), (120, 46))

In [32]:
_logits = np.exp(_output_logits) / np.sum(np.exp(_output_logits), 2)[:, :, None]

In [33]:
_idxs = _dec_target_idxs_do

In [35]:
_losses = np.array([[-np.log(_logits[i, j, _idxs[i, j]]) for j in range(_idxs.shape[1])] for i in range(_idxs.shape[0])]) * _dec_mask_tokens_do

In [36]:
np.sum(_losses)/np.sum(_dec_mask_tokens_do)

9.903732

In [37]:
_recon_loss

9.903732

In [38]:
_kl_losses.shape

(120,)